<a href="https://colab.research.google.com/github/Ammar-Kurt/Data_Structures/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install langchain_openai
%pip install langchain_community
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import openai
from dotenv import load_dotenv
import os
import shutil



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [21]:
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']

CHROMA_PATH = "chromas"
DATA_PATH = "/content/Research_papers"


In [22]:
%pip install fitz
%pip install PyMuPDF
import os
import fitz


class CustomFileLoader:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def load(self):
        documents = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if os.path.isfile(file_path):
                if filename.lower().endswith('.txt'):
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                        documents.append(Document(page_content=content))
                    except UnicodeDecodeError:
                        print(f"Error decoding file {file_path}. Skipping this file.")
                        continue
                elif filename.lower().endswith('.pdf'):
                    try:
                        pdf_document = fitz.open(file_path)
                        pdf_text = ""
                        for page_num in range(len(pdf_document)):
                            pdf_text += pdf_document[page_num].get_text()
                        documents.append(Document(page_content=pdf_text))
                    except Exception as e:
                        print(f"Error reading PDF file {file_path}: {e}. Skipping this file.")
                        continue
        return documents


def main():
    try:
        generate_data_store()
    except PermissionError as e:
        print(f"PermissionError occurred: {e}")
        # Handle or log the error appropriately
    except Exception as e:
        print(f"Error occurred in main(): {e}")

def generate_data_store():
    try:
        documents = load_documents()
        chunks = split_text(documents)
        save_to_chroma(chunks)
    except FileNotFoundError as e:
        print(f"FileNotFoundError occurred: {e}")
        # Handle or log the error appropriately
    except Exception as e:
        print(f"Error occurred in generate_data_store(): {e}")

def load_documents():
    try:
        loader = CustomFileLoader(DATA_PATH)
        documents = loader.load()
        return documents
    except FileNotFoundError as e:
        print(f"FileNotFoundError occurred: {e}")
        # Handle or log the error appropriately
    except PermissionError as e:
        print(f"PermissionError occurred: {e}")
        # Handle or log the error appropriately
    except Exception as e:
        print(f"Error occurred in load_documents(): {e}")

def split_text(documents):
    try:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=100,
            length_function=len,
            add_start_index=True
        )
        chunks = text_splitter.split_documents(documents)
        print(f"Split {len(documents)} documents into {len(chunks)} chunks")

        # Example usage: printing content and metadata of a chunk
        document = chunks[10]
        print(document.page_content)
        print(document.metadata)

        return chunks
    except Exception as e:
        print(f"Error occurred in split_text(): {e}")



In [23]:
def save_to_chroma(chunks):
    try:
        if os.path.exists(CHROMA_PATH):
            shutil.rmtree(CHROMA_PATH)

        # Create a Chroma object with a single file
        db = Chroma.from_documents(
            chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
        )
        db.persist()
        print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")
    except PermissionError as e:
        print(f"PermissionError occurred: {e}")
        # Handle or log the error appropriately
    except Exception as e:
        print(f"Error occurred in save_to_chroma(): {e}")

if __name__ == "__main__":
    main()

Split 3 documents into 1095 chunks
© The Author(s) 2023 
N. Gerber et al. (eds.), Human Factors in Privacy Research, 
https://doi.org/10.1007/978-3-031-28643-8_7
137
138
J. Johansen and S. Fischer-Hübner
First, we validate a rather general idea, of the type often found in position papers,
{'start_index': 1769}
Saved 1095 chunks to chromas


In [25]:
import argparse
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

CHROMA_PATH = "/content/chromas"

def main():
  parser = argparse.ArgumentParser()
  parser.add_argument("query_text",type=str, help="The query text")
  args = parser.parse_args()
  query_text = args.query_text

  embedding_function = OpenAIEmbeddings()
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

  results = db.similarity_search_with_relevance_scores(query_text,k=3)
  if len(results) == 0 or results[0][1] < 0.7:
   print(f"Unable to find matching results")
   return

  context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  print(prompt)
  model = ChatOpenAI(model='gpt-3.5-turbo')
  response_text = model.predict(prompt)
  sources = [doc.metadata.get("source", None) for doc, _score in results]
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  print(formatted_response)


if __name__ == "__main__":
    main()



usage: colab_kernel_launcher.py [-h] query_text
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [28]:
%run kernal.py "I have decided to make a AI driven solution for ALexithymic patients. is it a validayed idea?"


Human: 
Answer the question based only on the following context:

indications with practical AI systems, as they are intended to
function with little to no human intervention or even to tune their
behaviour autonomously in response to the environment (Vas-
sev and Hinchey, 2014). Moreover, measures may be needed

---

in [S8], a service robot for an elderly care centre was initially
validated in a laboratory setting with specific tasks before vali-
dating it in its real deployment environment. It was instructed to
enter a room, to locate and approach a person, and to recognize

---

are interested in research concerning practical AI systems that are
validated using one or more methods, and, on the other hand,
research on validation method proposals that are applied to some
practical system. Thus, solution proposals, ideas, and theoretical

---

synthesization, assessment, and refinement of AI system validation in research and guidelines for
validating individual systems in practice. Wh